In [10]:
import pandas as pd
from qlib.data.dataset.processor import RobustZScoreNorm, Fillna, DropnaLabel, CSZScoreNorm

In [12]:
import warnings  
warnings.filterwarnings("ignore", category=FutureWarning) 

In [5]:
df2 = pd.read_parquet('D:\CX_code\Graph series\MASTER\data\\000300.XSHG_component.pq')
print(df2)

ticker      000001.XSHE  000002.XSHE  000008.XSHE  000009.XSHE  000027.XSHE  \
date                                                                          
2016-01-04          1.0          1.0          NaN          1.0          1.0   
2016-01-05          1.0          1.0          NaN          1.0          1.0   
2016-01-06          1.0          1.0          NaN          1.0          1.0   
2016-01-07          1.0          1.0          NaN          1.0          1.0   
2016-01-08          1.0          1.0          NaN          1.0          1.0   
...                 ...          ...          ...          ...          ...   
2024-06-26          1.0          1.0          NaN          NaN          NaN   
2024-06-27          1.0          1.0          NaN          NaN          NaN   
2024-06-28          1.0          1.0          NaN          NaN          NaN   
2024-07-01          1.0          1.0          NaN          NaN          NaN   
2024-07-02          1.0          1.0          NaN   

In [11]:
df_test = pd.read_parquet('combined_data.pq')

# step2:设置双重列索引
level0 = ['feature'] * 44 + ['label']  # 第一层索引：前44列为'feature'，最后一列为'label'
level1 = list(df_test.keys())
multi_index = pd.MultiIndex.from_tuples(list(zip(level0, level1)))
df_test.columns = multi_index
# print(df_test.head())

# step3:设置双重行索引
# print("Original index names:", df_test.index.names)
df_test.index.names = ['datetime', 'instrument']
# print("New index names:", df_test.index.names)
# print(df_test.index)
df_sorted = df_test.sort_index()
# print(df_sorted.index)
df_sorted = df_sorted[df_sorted['label']["label"] != 0]

# step4:数据预处理
# print(df_test.index.get_level_values(0).unique())
# row_counts = df_test.groupby(level=0).size()
# print(row_counts)
RobustZScoreNorm = RobustZScoreNorm(fit_start_time='20160101', fit_end_time='20170501', fields_group='feature', clip_outlier=True)
Fillna = Fillna(fields_group='feature')
CSZScoreNorm = CSZScoreNorm(fields_group='label')

RobustZScoreNorm.fit(df_sorted)
for process_func in [RobustZScoreNorm, Fillna, CSZScoreNorm]:
    df_process = process_func(df_sorted)

C:\Users\Administrator\.conda\envs\Master\lib\site-packages\qlib\data\dataset\processor.py:299: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df[cols] = df[cols].groupby("datetime").apply(lambda x: (x - x.mean()).div(x.std()))


In [15]:
df1 = df_process
print(df1)

                                           feature                 \
                       240124_ret_buy_ols_beta_min 240129_avt_umr   
datetime   instrument                                               
2016-01-07 000001.XSHE                    0.000000       0.000000   
           000004.XSHE                    0.000000       0.000000   
           000005.XSHE                    0.000000       0.000000   
           000006.XSHE                    0.000000       0.000000   
           000008.XSHE                    0.000000       0.000000   
...                                            ...            ...   
2017-12-29 603991.XSHG                    3.000000      -0.981840   
           603993.XSHG                   -1.311337      -2.012916   
           603996.XSHG                    0.104988       3.000000   
           603997.XSHG                    3.000000       0.317388   
           603999.XSHG                    3.000000      -0.314721   

                                 

In [26]:
df1.index.names

FrozenList(['datetime', 'instrument'])

In [35]:
hs300_history = pd.DataFrame(columns=df1.columns, index=pd.MultiIndex(levels=[[], []], codes=[[], []], names=df1.index.names))

In [44]:
for date in df2.index:  
    # 检查日期是否存在于df1的索引中  
    if date not in df1.index.levels[0]:  
        continue  # 如果日期不在df1的索引中，则跳过这个日期  
    # 获取当前日期属于沪深300的股票列表  
    hs300_stocks = df2.loc[date].dropna().index.tolist()  
    # 使用股票列表从df1中过滤出相应股票的历史数据  
    date_data = df1.loc[(date, hs300_stocks), :]
    hs300_history = pd.concat([hs300_history, date_data]) 

In [45]:
print(hs300_history)

                                           feature                 \
                       240124_ret_buy_ols_beta_min 240129_avt_umr   
datetime   instrument                                               
2016-01-07 000001.XSHE                    0.000000       0.000000   
           000009.XSHE                    0.000000       0.000000   
           000027.XSHE                    0.000000       0.000000   
           000039.XSHE                    0.000000       0.000000   
           000046.XSHE                    0.000000       0.000000   
...                                            ...            ...   
2017-12-29 603160.XSHG                   -0.590650      -1.833890   
           603799.XSHG                   -1.286818      -0.722503   
           603833.XSHG                   -0.123341       0.485278   
           603858.XSHG                   -0.332305       0.864540   
           603993.XSHG                   -1.311337      -2.012916   

                                 

[145500 rows x 45 columns]
[266294 rows x 45 columns]